## Importing libraries

In [149]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
import emoji

In [150]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading datasets

In [151]:
df_1 = pd.read_csv('db_AGP_2019_sucesos.csv')
df_2 = pd.read_csv('postID_suceso.csv')

## Exploring dataset

In [152]:
df_1.head()

,created_time.1,created_time.2,message,id,object_link.connections.comments.message,FECHA Y HORA
0,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🧐🧐,19/04/2019 9:16
1,"jueves, 18 de abril de 2019",30/12/1899 20:57:16,"Exequias del ex presidente Alan García en ""La ...",71263708835_276236269920595,🥺🥺🥺🥺🥺🥺🥺🥺,18/04/2019 20:57
2,"miércoles, 17 de abril de 2019",30/12/1899 07:37:30,🚨 Estamos en los exteriores del Hospital de Em...,71263708835_310239669653341,🥳,17/04/2019 7:37
3,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤷‍♀️🤷‍♀️🤷‍♀️,19/04/2019 9:16
4,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤯🤯🤯🤯,19/04/2019 9:16


In [153]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55006 entries, 0 to 55005
Data columns (total 6 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   created_time.1                            55006 non-null  object
 1   created_time.2                            55006 non-null  object
 2   message                                   55006 non-null  object
 3   id                                        55006 non-null  object
 4   object_link.connections.comments.message  55006 non-null  object
 5   FECHA Y HORA                              55006 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [154]:
df_2.head()

,POST NRO,SUCESO
0,71263708835_310239669653341,Traslado a la Clinica
1,71263708835_10157689377373836,Confirmación de Intento de Suicidio
2,71263708835_2430897753596693,Confirmación de Intento de Suicidio
3,71263708835_10157689601593836,Confirmación de Intento de Suicidio
4,71263708835_426911824739777,Confirmación de Intento de Suicidio


In [155]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   POST NRO  87 non-null     object
 1   SUCESO    87 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


## Cleaning and Tranform

In [156]:
# drop cols
df_1.drop(columns=['created_time.2','created_time.1'],inplace=True)

# merge the both datasets
df = df_1.merge(df_2,how='left',left_on = 'id',right_on='POST NRO')

In [157]:
# drop redundant cols
df.drop(columns=['id','POST NRO'],inplace=True)

In [158]:
# change the names of the columns
df.columns = ['post','comment','datetime','context']

# order the columns
df = df[['datetime','context','post','comment']]

In [159]:
# change the type of 'datetime'
df['datetime'] = pd.to_datetime(df['datetime'],format='%d/%m/%Y %H:%M')

In [160]:
# check nan values
df.isnull().sum()

datetime    0
context     0
post        0
comment     0
dtype: int64

In [161]:
# check duplicated values
df.duplicated().sum()

1444

In [162]:
# drop duplicated values
df.drop_duplicates(inplace=True)

In [163]:
punct = string.punctuation
df['comment'] = df['comment'].str.replace(f'[{punct}]','',regex=True).str.lower()

In [164]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [165]:
import unicodedata

def eliminar_tildes(texto):
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    texto_sin_tildes = ''.join(c for c in texto_normalizado if not unicodedata.combining(c))
    return texto_sin_tildes

In [166]:
df['comment'] = df['comment'].apply(remove_stopwords)
df['comment'] = df['comment'].apply(eliminar_tildes)

In [167]:
df.reset_index(drop=True, inplace=True)

In [168]:
emoji_to_spanish = {
    "©": "copyright ",
    "®": "registrado ",
    "‼": "sorpresa ",
    "⁉": "duda ",
    "▪": "cuadro negro ",
    "☀": "sol ",
    "☕": "café ",
    "☝": "arriba ",
    "☠": "peligro ",
    "☹": "triste ",
    "☺": "feliz ",
    "♀": "mujer ",
    "♂": "hombre ",
    "♥": "amor ",
    "⚖": "justicia ",
    "⚠": "advertencia ",
    "⚰": "ataúd ",
    "⚽": "fútbol ",
    "⛏": "pico ",
    "⛪": "iglesia ",
    "✅": "verificado ",
    "✈": "avión ",
    "✊": "puño ",
    "✋": "mano ",
    "✌": "paz ",
    "✍": "escribir ",
    "✔": "correcto ",
    "✝": "cruz ",
    "✨": "brillo ",
    "❌": "incorrecto ",
    "❎": "cancelado ",
    "❗": "exclamación ",
    "❤": "corazón ",
    "➕": "más ",
    "⬆": "arriba ",
    "⬇": "abajo ",
    "⭐": "estrella ",
    "〰": "línea_ondulada ",
    "🅱": "b ",
    "🆘": "emergencia ",
    "🌊": "ola ",
    "🌌": "galaxia ",
    "🌍": "tierra ",
    "🌏": "planeta ",
    "🌞": "sol ",
    "🌟": "estrella ",
    "🌠": "estrella fugaz ",
    "🌱": "planta ",
    "🌶": "picante ",
    "🌷": "flor ",
    "🌹": "rosa ",
    "🌻": "girasol ",
    "🌿": "hierba ",
    "🍃": "hoja ",
    "🍒": "cereza ",
    "🍯": "miel ",
    "🍺": "cerveza ",
    "🍻": "brindis ",
    "🍾": "champan ",
    "🍿": "palomitas ",
    "🎆": "fuegos artificiales ",
    "🎇": "chispa ",
    "🎉": "celebración ",
    "🎊": "fiesta ",
    "🎓": "graduación ",
    "🎛": "controles ",
    "🎥": "cine ",
    "🎪": "circo ",
    "🎬": "claqueta ",
    "🎭": "teatro ",
    "🎵": "nota ",
    "🎶": "música ",
    "🎼": "partitura ",
    "🏃": "correr ",
    "🏆": "trofeo ",
    "🏌": "golf ",
    "🏙": "ciudad ",
    "🏝": "isla ",
    "🏡": "casa ",
    "🏥": "hospital ",
    "🏻": "piel_clara ",
    "🏼": "piel_media_clara ",
    "🏽": "piel_media ",
    "🏾": "piel_media_oscura ",
    "🏿": "piel_oscura ",
    "🐀": "rata ",
    "🐁": "ratón ",
    "🐈": "gato ",
    "🐎": "caballo ",
    "🐑": "oveja ",
    "🐖": "cerdo ",
    "🐗": "jabalí ",
    "🐤": "pollito ",
    "🐥": "pollito_feliz ",
    "🐭": "ratón_dibujo ",
    "🐱": "gato_dibujo ",
    "🐴": "caballo_cabeza ",
    "🐶": "perro ",
    "🐷": "cerdo_cabeza ",
    "🐺": "lobo ",
    "👀": "ojos ",
    "👁": "ojo ",
    "👂": "oído ",
    "👅": "lengua ",
    "👆": "arriba ",
    "👇": "abajo ",
    "👈": "izquierda ",
    "👉": "derecha ",
    "👊": "puño_cerrado ",
    "👋": "saludo ",
    "👌": "ok ",
    "👍": "pulgar_arriba ",
    "👎": "pulgar_abajo ",
    "👏": "aplauso ",
    "👐": "manos_abiertas ",
    "👚": "blusa ",
    "👝": "bolso ",
    "👨": "hombre ",
    "👪": "familia ",
    "👹": "ogro ",
    "👺": "demonio ",
    "👻": "fantasma ",
    "👼": "ángel ",
    "👽": "alien ",
    "👾": "extraterrestre ",
    "👿": "diablo ",
    "💀": "calavera ",
    "💁": "persona_indicando ",
    "💃": "bailando ",
    "💆": "masaje ",
    "💐": "flores ",
    "💒": "boda ",
    "💓": "corazón_latente ",
    "💔": "corazón_roto ",
    "💕": "corazones ",
    "💖": "brillo_corazón ",
    "💗": "corazón_grande ",
    "💘": "corazón_flecha ",
    "💚": "corazón_verde ",
    "💜": "corazón_morado ",
    "💡": "idea ",
    "💣": "bomba ",
    "💤": "dormir ",
    "💥": "explosión ",
    "💦": "gotas ",
    "💨": "viento ",
    "💩": "popó ",
    "💪": "fuerza ",
    "💫": "mareo ",
    "💭": "pensamiento ",
    "💯": "perfecto ",
    "💰": "dinero ",
    "💲": "dólar ",
    "💳": "tarjeta ",
    "💵": "billete ",
    "💶": "euro ",
    "💷": "libra ",
    "💹": "crecimiento ",
    "📢": "megáfono ",
    "📣": "altavoz ",
    "📸": "foto ",
    "📺": "televisión ",
    "🔊": "volumen_alto ",
    "🔐": "cerrado ",
    "🔥": "fuego ",
    "🔨": "martillo ",
    "🔪": "cuchillo ",
    "🔫": "pistola ",
    "🔱": "tridente ",
    "🔴": "círculo_rojo ",
    "🔵": "círculo_azul ",
    "🔷": "rombo_azul ",
    "🕊": "paloma ",
    "🕯": "vela ",
    "🕵": "detective ",
    "🖐": "mano ",
    "🖕": "dedo_medio ",
    "🖤": "corazón negro ",
    "🗑": "basura ",
    "🗼": "torre ",
    "😀": "sonrisa ",
    "😁": "carcajada ",
    "😂": "risa_llorando ",
    "😃": "alegría ",
    "😄": "risa_grande ",
    "😅": "alivio ",
    "😆": "risa_abierta ",
    "😇": "angelito ",
    "😈": "diablillo ",
    "😉": "guiño ",
    "😊": "contento ",
    "😋": "sabroso ",
    "😌": "tranquilo ",
    "😍": "enamorado ",
    "😎": "cool ",
    "😏": "satisfecho ",
    "😐": "neutral ",
    "😑": "indiferente ",
    "😒": "decepcionado ",
    "😓": "sudor ",
    "😔": "pensativo ",
    "😕": "confuso ",
    "😖": "preocupado ",
    "😗": "beso ",
    "😘": "beso_amor ",
    "😙": "beso_feliz ",
    "😚": "beso_cerrado ",
    "😛": "lengua_fuera ",
    "😜": "lengua_guiño ",
    "😝": "lengua_broma ",
    "😞": "frustrado ",
    "😟": "angustiado ",
    "😠": "enojado ",
    "😡": "furioso ",
    "😢": "llorar ",
    "😣": "perseverante ",
    "😤": "determinación ",
    "😥": "aliviado ",
    "😦": "preocupación ",
    "😧": "shock ",
    "😨": "temeroso ",
    "😩": "agotado ",
    "😪": "somnoliento ",
    "😫": "agotamiento ",
    "😬": "incomodo ",
    "😭": "llorar_mucho ",
    "😮": "sorpresa ",
    "😯": "silencio ",
    "😰": "ansiedad ",
    "😱": "grito ",
    "😲": "impactado ",
    "😳": "avergonzado ",
    "😴": "dormido ",
    "😵": "mareado ",
    "😶": "sin_palabras ",
    "😷": "enfermo ",
    "😸": "gato_feliz ",
    "😹": "gato_risa ",
    "😼": "gato_malicioso ",
    "😿": "gato_triste ",
    "🙁": "decepcionado ",
    "🙂": "feliz ",
    "🙃": "al_reves ",
    "🙄": "ojo_volteado ",
    "🙅": "prohibido ",
    "🙆": "ok ",
    "🙇": "reverencia ",
    "🙈": "no_ver ",
    "🙉": "no_escuchar ",
    "🙊": "no_hablar ",
    "🙋": "mano_alzada ",
    "🙌": "victoria ",
    "🙏": "oración ",
    "🚆": "tren ",
    "🚑": "ambulancia ",
    "🚫": "prohibido ",
    "🚶": "caminar ",
    "🛥": "barco ",
    "🛩": "avión ",
    "🛫": "despegue ",
    "🛬": "aterrizaje ",
    "🛸": "ovni ",
    "🤐": "silencio ",
    "🤑": "dinero ",
    "🤒": "enfermo ",
    "🤓": "nerd ",
    "🤔": "pensativo ",
    "🤕": "herido ",
    "🤗": "abrazo ",
    "🤘": "rock ",
    "🤙": "shaka ",
    "🤞": "cruzar_dedos ",
    "🤟": "te_amo ",
    "🤡": "payaso ",
    "🤢": "asco ",
    "🤣": "carcajada ",
    "🤤": "baba ",
    "🤥": "mentira ",
    "🤦": "frustración ",
    "🤧": "estornudo ",
    "🤨": "sospecha ",
    "🤩": "estrellas ",
    "🤪": "loco ",
    "🤫": "secreto ",
    "🤬": "molesto ",
    "🤭": "sorpresa ",
    "🤮": "vómito ",
    "🤯": "impacto ",
    "🤶": "señora_navidad ",
    "🤷": "duda ",
    "🤹": "malabarista ",
    "🥀": "flor_muerta ",
    "🥂": "brindis ",
    "🥃": "trago ",
    "🥑": "aguacate ",
    "🥚": "huevo ",
    "🥳": "celebración ",
    "🥴": "mareado ",
    "🥵": "calor ",
    "🥶": "frío ",
    "🥺": "suplicar ",
    "🦁": "león ",
    "🦃": "pavo ",
    "🦆": "pato ",
    "🦊": "zorro ",
    "🦸": "héroe ",
    "🧐": "curioso ",
    "🧠": "cerebro ",
    "🧬": "adn ",
    "🧹": "escoba "
}

In [169]:
def clean_text_from_emojis(text):
    list_text_without_emojis = []
    list_text_with_emojis = []
    
    for char in text:
        if char not in emoji.EMOJI_DATA:
            list_text_without_emojis.append(char)
        else:
            list_text_with_emojis.append(char)
    return "".join(list_text_with_emojis), "".join(list_text_without_emojis)

In [170]:
new_comment = df['comment'].apply(clean_text_from_emojis)

In [171]:
emojis_in_comment, text_in_comment = [], []

for a,b in new_comment:
    emojis_in_comment.append(a)
    text_in_comment.append(b)

In [172]:
def translate_emojis_to_spanish(text):
    if not isinstance(text, str):
        return ""
    
    list_text = []
    for char in text:
        if char in emoji.EMOJI_DATA:
            list_text.append(emoji_to_spanish.get(char, "desconocido "))
        else:
            list_text.append(char)
    
    return "".join(list_text)

In [173]:
emojis_text = []

for text in emojis_in_comment:
    emojis_text.append(translate_emojis_to_spanish(text))

In [174]:
len(emojis_text)

53562

In [175]:
text_1

'– 1985 69 personas asesinadas accomarca – 1986 300 personas asesinadas lurigancho fronton – 1988 39 personas asesinadas cayara – 1989 62 indigenas junin incluyendo ninos ancianos mujeres asesinados acusandolos ser terroristas – 2006 2 personas mueren primero protesta minera yanacocha segundo protesta abancay – 2007 9 personas mueren dos consecuencia paro sutep 5 protestas ancash apurimac lima dos “error” fuerzas orden – 2008 37 personas asesinadas 3 pucallpa barranca ayacucho menos 34 asesinados “escuadrones muerte” trujillo – 2009 36 muertos 34 identificados mas 120 desaparecidos “baguazo” 24 policias 9 nativos 2 muertos enfrentamiento huancabamba – 2010 6 muertos chala represion estatal frente mineros artesanales alan garcia demuestrenlo pues imbeciles'

In [176]:
text = []
for text_1, text_2 in zip(text_in_comment, emojis_text):
    text.append(text_1+ ' '+ text_2)

In [177]:
df['comment 2'] = pd.Series(text)

In [178]:
df['comment 2'] = df['comment 2'].apply(eliminar_tildes)

## NRC predeterminado


In [179]:
import pandas as pd
lexicon = pd.read_csv('espaniol_NRC.csv', index_col='Spanish Word')
lexicon.drop(columns=['anticipacion','positivo','confianza','negativo'], inplace=True)


def feelings_in_text(text):
    feelings_count = {col: 0 for col in lexicon.columns}
    words = text.split()

    for word in words:
        if word in lexicon.index:
            feeling_values = lexicon.loc[word].values
            for i, value in enumerate(feeling_values):
                feelings_count[lexicon.columns[i]] += value

    feelings, count = sorted(feelings_count.items(), key=lambda item: item[1], reverse=True)[0]
    if count == 0:
        return 'neutral'
    else: 
        return feelings

In [180]:
temp_data = df['comment 2'].apply(feelings_in_text)

In [181]:
temp_data

0         neutral
1         neutral
2         alegria
3           miedo
4         neutral
           ...   
53557    disgusto
53558    disgusto
53559       miedo
53560       miedo
53561       miedo
Name: comment 2, Length: 53562, dtype: object

In [182]:
from collections import Counter

Counter(temp_data)

Counter({'neutral': 21884,
         'ira': 9774,
         'miedo': 8198,
         'disgusto': 7279,
         'alegria': 3881,
         'tristeza': 2168,
         'sorpresa': 378})

In [183]:
temp_data

0         neutral
1         neutral
2         alegria
3           miedo
4         neutral
           ...   
53557    disgusto
53558    disgusto
53559       miedo
53560       miedo
53561       miedo
Name: comment 2, Length: 53562, dtype: object

In [184]:
df['sentiments']= temp_data

In [185]:
df

,datetime,context,post,comment,comment 2,sentiments
0,2019-04-19 09:16:00,Lectura de Carta de AG,Velorio del ex presidente Alan García en la Ca...,🧐🧐,curioso curioso,neutral
1,2019-04-18 20:57:00,Velatorio Casa del Pueblo,"Exequias del ex presidente Alan García en ""La ...",🥺🥺🥺🥺🥺🥺🥺🥺,suplicar suplicar suplicar suplicar suplicar ...,neutral
2,2019-04-17 07:37:00,Traslado a la Clinica,🚨 Estamos en los exteriores del Hospital de Em...,🥳,celebracion,alegria
3,2019-04-19 09:16:00,Lectura de Carta de AG,Velorio del ex presidente Alan García en la Ca...,🤷‍♀️🤷‍♀️🤷‍♀️,‍️‍️‍️ duda mujer duda mujer duda mujer,miedo
4,2019-04-19 09:16:00,Lectura de Carta de AG,Velorio del ex presidente Alan García en la Ca...,🤯🤯🤯🤯,impacto impacto impacto impacto,neutral
...,...,...,...,...,...,...
53557,2019-04-18 22:01:00,Portadas del día,Hoy en portada de El Comercio ↓\r\n- Alan Garc...,comercio asi saques primera pagina dr alan gar...,comercio asi saques primera pagina dr alan gar...,disgusto
53558,2019-04-19 12:35:00,Ataud Sale del local APRA,El féretro que lleva el cuerpo del ex presiden...,xq nadie puede ver cadaver,xq nadie puede ver cadaver,disgusto
53559,2019-04-18 19:12:00,Velatorio Casa del Pueblo,"Decenas de simpatizantes del Partido Aprista, ...",– 1985 69 personas asesinadas accomarca – 1986...,– 1985 69 personas asesinadas accomarca – 1986...,miedo
53560,2019-04-18 19:12:00,Velatorio Casa del Pueblo,"Decenas de simpatizantes del Partido Aprista, ...",– 1985 69 personas asesinadas accomarca – 1986...,– 1985 69 personas asesinadas accomarca – 1986...,miedo


In [186]:
def cortar(text):
    text = str(text)
    return text[8:10]

df['day'] = df['datetime'].apply(cortar)

In [187]:
df.drop(columns=['post','comment'],inplace=True)

In [ ]:
df.to_csv('clean_data',index=False)

,datetime,context,comment 2,sentiments,day
0,2019-04-19 09:16:00,Lectura de Carta de AG,curioso curioso,neutral,19
1,2019-04-18 20:57:00,Velatorio Casa del Pueblo,suplicar suplicar suplicar suplicar suplicar ...,neutral,18
2,2019-04-17 07:37:00,Traslado a la Clinica,celebracion,alegria,17
3,2019-04-19 09:16:00,Lectura de Carta de AG,‍️‍️‍️ duda mujer duda mujer duda mujer,miedo,19
4,2019-04-19 09:16:00,Lectura de Carta de AG,impacto impacto impacto impacto,neutral,19
...,...,...,...,...,...
53557,2019-04-18 22:01:00,Portadas del día,comercio asi saques primera pagina dr alan gar...,disgusto,18
53558,2019-04-19 12:35:00,Ataud Sale del local APRA,xq nadie puede ver cadaver,disgusto,19
53559,2019-04-18 19:12:00,Velatorio Casa del Pueblo,– 1985 69 personas asesinadas accomarca – 1986...,miedo,18
53560,2019-04-18 19:12:00,Velatorio Casa del Pueblo,– 1985 69 personas asesinadas accomarca – 1986...,miedo,18


In [107]:
pd.crosstab(df['day'],df['context'])

context,Ataud Sale del local APRA,Confirmación de Intento de Suicidio,Confirmación de Muerte,Lectura de Carta de AG,Portadas del día,Traslado a la Clinica,Velatorio Casa del Pueblo
day,,,,,,,
17,0,5666,11225,0,0,2841,0
18,0,0,4389,0,783,0,13600
19,3289,0,0,9225,2037,0,0
20,507,0,0,0,0,0,0
